<a href="https://colab.research.google.com/github/Ali-mohammadi-design/EBD-TRIZ-LLM/blob/main/EBD_TRIZ_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain==0.0.307
!pip install openai==0.27.0
!pip install tiktoken==0.3.0
!pip install wikipedia
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.0
    Uninstalling langsmith-0.3.0:
      Successfully uninstalled langsmith-0.3.0
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.15
    Uninstalling langchain-0.3.15:
      Successfully uninstalled langchain-0.3.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-core 0.3.31 requires langsmith<0.4,>=0.1.125, but you have langsmith 0.0.92 which is incomp

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
import os
openAI_API_Key=os.environ['openAI_API_Key']='Your API Key'

In [ ]:
llm= ChatOpenAI(openai_api_key=openAI_API_Key,temperature=0)

**Step1: Asking Questions**

In [ ]:
json_schema={'title':'questions related to the verbs',
             'description':'questions about the verbs of the sentences',
             'type':'object',
             'properties':{
                 'question with the why': {'title':'question with the why','description':'ask why from verb of the sentence','data_type':'string'},
                 'question with the where': {'title':'question with the where','description':'ask where from verb of the sentence','data_type':'string'},
                 'question with the when': {'title':'question with the when','description':'ask when from verb of the sentence','data_type':'string'},
                 'question with the how': {'title':'question with the how','description':'ask how from verb of the sentence','data_type':'string'},
                 'question with the who': {'title':'question with the who','description':'ask who from verb of the sentence','data_type':'string'}

             },

             'required':['question with the why','question with the where','question with the when','question with the how']
             }


In [ ]:
def question_asking(sentence):
  #First section: Generating questions for the nouns:

  #In the first part of the process, the function extracts the nouns from the input text.
  system_template1=" You are an AI assistant that gets a text and returns all nouns, pronouns, and noun phrases. Please recognize verbs but do not return verbs or propositions"
  system_message_prompt1= SystemMessagePromptTemplate.from_template(system_template1)
  human_template1='please list all the pronouns, nouns, and noun phrases  in the following text: \n {text}'
  human_message_prompt1=HumanMessagePromptTemplate.from_template(human_template1)
  prompt1=ChatPromptTemplate.from_messages([system_message_prompt1,human_message_prompt1])
  chain1=LLMChain(llm=llm, prompt=prompt1, output_key='nouns')

#In the following section, we will determine whether this noun refers to a human or a non-human entity.
  human_template2='Consider the following list of nouns,pronouns,and nouns phrase. please specify which ones are human and which ones are non-human.This is the list: {nouns}'
  prompt2=ChatPromptTemplate.from_template(human_template2)
  chain2=LLMChain(llm=llm, prompt=prompt2, output_key='type')

# The following code generates questions with 'what' or 'who' for every noun.
  system_template3=''' you are an AI assistant that get nouns,pronouns,and nouns phrases and  their type and ask question with what or who.
  for the human nouns,pronouns,and nouns-phrases you ask: who is + (nouns,pronouns,and nouns phrases)? \n For non-human ones, you ask: what is + (nouns,pronouns,and nouns phrases)?'''
  system_message_prompt3= SystemMessagePromptTemplate.from_template(system_template3)
  human_template3='''please consider the folowing nouns. for each noun you must ask questions with what or who depends on their type.
  for example if the nouns are teacher and sandwich you shoud return who is teacher? and what is sandwich? The nouns are : \n {type}'''
  human_message_prompt3=HumanMessagePromptTemplate.from_template(human_template3)
  noun1='Bob, school, sandwich'
  example_one= HumanMessagePromptTemplate.from_template(noun1)
  question1='who is Bob? \n what is school? \n what is sandwich?'
  example_one_output= AIMessagePromptTemplate.from_template(question1)
  prompt3=ChatPromptTemplate.from_messages([system_message_prompt3,human_message_prompt3])
  chain3=LLMChain(llm=llm, prompt=prompt3, output_key='questions')

#Executing the chain
  text=sentence
  seq_chain=SequentialChain(chains=[chain1,chain2,chain3], input_variables=['text'],output_variables=['nouns','type','questions'],verbose=False)
  result1=seq_chain(text)
  noun_questions=result1


#Second section: Generating questions for the verbs

# The first part of this section would separate the text into sentences.
  system_template_verb_0=''' "You are an AI assistant who aims to get a text and separate its sentences.
  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template_verb_0)
  human_template_verb_0='''' "I'll provide a text for you. Please examine the given text, first identifying its verb phrases.
  After that, take note of each verb phrase you've identified in the provided text and return separate sentences for each verb phrase that include subject + verb phrase + object.
  \n The sentence is: \n {text}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template_verb_0)
  prompt_verb_0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  chain_verb_0=LLMChain(llm=llm, prompt=prompt_verb_0, output_key='all_sentences')

# The second part of this section would consider each sentence and generate various questions with 4 wh questions and with each separate sentence.
  system_template_verb_2=''' You are an AI assistant who aims to help designers to ask question.
  '''
  system_message_prompt_verb_2= SystemMessagePromptTemplate.from_template(system_template_verb_2)
  human_template_verb_2=''''
  I will provide you with a text containing subjects, verbs, objects and the sentences. Your task is to ask questions.
  generate five questions for each sentence using 'where, why, how, when, who.
  You must follow this structure:'wh question' + 'auxiliary verb' + 'subject' + 'verb' + 'object' + '?'.
  This means that for each sentence, you will create one question for each of the five question words.
  \n\nThe text to be used for this exercise is:\n\n {all_sentences}"
  '''
  human_message_prompt_verb_2=HumanMessagePromptTemplate.from_template(human_template_verb_2)
  prompt_verb_2=ChatPromptTemplate.from_messages([system_message_prompt_verb_2,human_message_prompt_verb_2])
  chain_verb_2=LLMChain(llm=llm, prompt=prompt_verb_2, output_key='all_questions')

#Executing the chain of this section
  seq_chain2=SequentialChain(chains=[chain_verb_0,chain_verb_2], input_variables=['text'],output_variables=['all_sentences','all_questions'],verbose=False)
  result2=seq_chain2(text)
  verb_questions=result2

# The text will be modified to remove extra explanation and only include questions.
  template_pars_out= "Please return only the questions from the following passage. \n\n this is the text:\n\n {undesired_output}"
  chat_prompt_pars_out_template=ChatPromptTemplate.from_template(template_pars_out)
  chat_prompt=ChatPromptTemplate.from_messages([chat_prompt_pars_out_template])
  request_parsout=chat_prompt.format_prompt(undesired_output=verb_questions['all_questions']).to_messages()
  result_pars_out_final=llm(request_parsout)


#Third Section: Generating questions related to the meaning of verbs.

#Extracting the verbs
  system_template_verb_meaning_0=" You are an AI assistant that gets text. You get some text. first recoginize all the sentences in the text. Then for each sentence return the verb"
  system_message_prompt_verb_meaning_0= SystemMessagePromptTemplate.from_template(system_template_verb_meaning_0)
  human_template_verb_meaning_0='please first consider the sentences in this text and then list the vebr in each sentence: \n {all_sentences}'
  human_message_prompt_verb_meaning_0=HumanMessagePromptTemplate.from_template(human_template_verb_meaning_0)
  prompt_verb_meaning_0=ChatPromptTemplate.from_messages([system_message_prompt_verb_meaning_0,human_message_prompt_verb_meaning_0])
  chain_verb_meaning_0=LLMChain(llm=llm, prompt=prompt_verb_meaning_0, output_key='verb_list')

#Asking question with "what do you mean by + the verb?" template.
  system_template_verb_meaning_1=" you are an AI assistant that get verbs and ask question with what.you ask: what do you mean by + the verb?"
  system_message_prompt_verb_meaning_1= SystemMessagePromptTemplate.from_template(system_template_verb_meaning_1)
  human_template_verb_meaning_1='''please consider the folowing verbs. for each verb you must ask questions with what.
  for example, if the verb is eat you should return: what do you mean by eating? \n\n list of verbs are: {verb_list}'''
  human_message_prompt_verb_meaning_1=HumanMessagePromptTemplate.from_template(human_template_verb_meaning_1)
  prompt_verb_meaning_1=ChatPromptTemplate.from_messages([system_message_prompt_verb_meaning_1,human_message_prompt_verb_meaning_1])
  chain_verb_meaning_1=LLMChain(llm=llm, prompt=prompt_verb_meaning_1, output_key='verb_meaning_questions')

#Executing the chain
  text=sentence
  seq_chain3=SequentialChain(chains=[chain_verb_meaning_0,chain_verb_meaning_1], input_variables=['all_sentences'],output_variables=['verb_list','verb_meaning_questions'],verbose=False)
  result3=seq_chain3(text)
  verbs_meaning=result3


#Fourth section: Asking question from the whole design problem

  system_template0=''' You are an AI assistant who aims to help designers to extract as much information as possible from a design problem.
  You get a design problem.
  You should ask six questions with the following format:
  1- what do you mean by + the whole design problem +?
  2- Where do you need + the whole design problem + ?
  3- when do you need + the whole design problem + ?
  4- why do you need + the whole design problem + ?
  5- How do you need + the whole design problem + ?
  6- who is responsible for + the whole design problem + ?

  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template0)
  human_template0=''''
  \n The design problem is: \n {text}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template0)
  prompt0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  request=prompt0.format_prompt(text=text).to_messages()
  result0=llm(request)
  verb_questions2=result0


#Collecting all the generated questions as a list in a text string
  question_final_list= f"\n \n {noun_questions['questions']}   \n\n{verbs_meaning['verb_meaning_questions']}  \n \n{ result_pars_out_final.content}\n\n {verb_questions2.content} "

#Fifth section: Eliminating duplicated questions and providing  unique questions

  system_template0=''' You are an AI assistant who can recognize repetetive concepts.
  You will receive a list of questions, some of these questions may be repetitive or asking about similar concept.
  Your objective is to provide a list of unique questions without modifying the wording of the original questions or making any other changes to them.

  '''
  system_message_prompt0= SystemMessagePromptTemplate.from_template(system_template0)
  human_template0=''''
  \n This is the list of questions: \n {question_final_list}"
  '''
  human_message_prompt0=HumanMessagePromptTemplate.from_template(human_template0)
  prompt0=ChatPromptTemplate.from_messages([system_message_prompt0,human_message_prompt0])
  request=prompt0.format_prompt(question_final_list=question_final_list).to_messages()
  result0=llm(request)
  question_final_list_unique=result0

#printing unique questions
  return print(f"these are the questions should be answered:\n\n{question_final_list_unique.content} \n\n\n...........\n\n The preliminary questions:\n\n{question_final_list}")


In [ ]:
question_asking('Design a passenger cabin that includes maximum number of seats while ensuring passenger comfort.')

these are the questions should be answered:

1. Who is passenger?
2. What is cabin?
3. What is number?
4. What is seats?
5. What is comfort?
6. What is maximum number of seats?
7. What do you mean by designing?
8. What do you mean by including?
9. What do you mean by ensuring?
10. Where do you ensure passenger comfort?
11. Why do you ensure passenger comfort?
12. How do you ensure passenger comfort?
13. When do you ensure passenger comfort?
14. Who ensures passenger comfort?
15. What do you mean by "design a passenger cabin that includes the maximum number of seats while ensuring passenger comfort"?
16. Where do you need a passenger cabin that includes the maximum number of seats while ensuring passenger comfort?
17. When do you need a passenger cabin that includes the maximum number of seats while ensuring passenger comfort?
18. Why do you need a passenger cabin that includes the maximum number of seats while ensuring passenger comfort?
19. How do you need a passenger cabin that inclu

**Second Stage: Answering the questions**

In [ ]:
# We need to create a class that can save stages of the lifecycle.
class object_lifecycle():
  def __init__(self,First_stage,Second_stage,Third_stage,Fourth_stage):
     self.first_stage =First_stage
     self.Second_stage=Second_stage
     self.Third_stage=Third_stage
     self.Fourth_stage=Fourth_stage

In [ ]:
#We need GPT to provide the life cycle of objects in 4 stages, in json format without extra explanation, for efficient environment generation.
json_schema={'title':'scientist',
             'description':'four stages of the creation of the object',
             'type':'object',
             'properties':{
                 'first_stage': {'title':'first_stage','description':'The First stage of the lifecycle of this object','data_type':'string'},
                 'second_stage': {'title':'Second_stage','description':'The second stage of the lifecycle of this object','data_type':'string'},
                 'third_stage': {'title':'third_stage','description':'The third stage of the lifecycle of this object','data_type':'string'},
                 'last_stage': {'title':'last_stage','description':'The last stage of the lifecycle of this object','data_type':'string'},
             },

             'required':['first_stage','second_stage','thrid stage','last stage']
             }

In [ ]:
model= ChatOpenAI(openai_api_key=openAI_API_Key, temperature=0)

In [ ]:
def What_OR_Who_is(name):


#Clarifying the meaning of the object
  system_template=" you are a dictionary that could help designers to define {objects}. You would define {objects} clearly and briefly in one sentence"
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= ' What is {objects}?'
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  request=chat_prompt.format_prompt(objects=name).to_messages()
  definition=llm(request)


# Recognizing the object's lifecycle, in the next ection we will generate objects in the environment of each stage using JSON format output in this section.
  system_template=" you are an AI assistant that could help designers to understand the lifecycle of {objects}. You would explain the lifecycle of {objects} clearly and very breifly in just four stages and provide a  short explanation for every stage "
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= 'I would define lifecycle as events that something passes to reach the point that it is. For instance the lifecycle of an engineer is studying in school, getting good marks, going to colledge, going to university, finding an engineer job. What is life cycle of {objects}'
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  chain=create_structured_output_chain(json_schema,llm, chat_prompt,verbose=False)
  result=chain.run(objects=name)
  obj=object_lifecycle(result['first_stage'],result['second_stage'],result['third_stage'],result['last_stage'])

#Identifying the objects at every stage of their lifecycle
  system_template=''' Every {objects} is related to it's environment.
  we would have three types of environment: 1- built environment, 2-human environment 3- nature environment.
  Built environment: Every thing directly related to the {objects} and made by human.
  Human environment: Every human being directly related to the {objects}.
  Nature environment: Every natural things like weather, water etc that is directly connect to {objects}.
  When we wnat to describe the environment of {objects} we have to show the relation between {objects} and the environment in senctnces.
  you are an AI assistant that could help designers to understand the environment of {objects}.
  You would explain the environment of {objects} with single sentences.You would prefer to use action verbs in the sentences'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
  human_template= '''{lifecycle_stage} is one of the stages of lifecycle of {objects}.
   Consider {objects} in this stage and determine every part of the environment of {objects} in the separate sentences'''
  human_prompt= HumanMessagePromptTemplate.from_template(human_template)
  chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt, human_prompt])
  model_request= chat_prompt.format_prompt(lifecycle_stage=obj.first_stage, objects=name).to_messages()
  env_in_first_stage=model(model_request)

  model_request2= chat_prompt.format_prompt(lifecycle_stage=obj.Second_stage, objects=name).to_messages()
  env_in_second_stage=model(model_request2)

  model_request3= chat_prompt.format_prompt(lifecycle_stage=obj.Third_stage, objects=name).to_messages()
  env_in_third_stage=model(model_request3)

  model_request4= chat_prompt.format_prompt(lifecycle_stage=obj.Fourth_stage, objects=name).to_messages()
  env_in_fourth_stage=model(model_request4)


#Printing the results
  result=f'''
  {name} definition: {definition.content}
  \n {name} usually has a specific life cycle and some environment in its lifecycle.
  \n The first stage of the life cycle of {name} is {obj.first_stage}. In this stage, the environment is classified as the following:\n{env_in_first_stage.content}
  \n The second stage of the life cycle of {name} is {obj.Second_stage}. In this stage, the environment is classified as the following:\n{env_in_second_stage.content}
  \n The third stage of the life cycle of {name} is {obj.Third_stage}. In this stage, the environment is classified as the following:\n{env_in_third_stage.content}
  \n The fourth stage of the life cycle of {name} is {obj.Fourth_stage}. In this stage, the environment is classified as the following:\n{env_in_fourth_stage.content}
  '''
  return print(result)

In [ ]:
What_OR_Who_is('design')


  design definition: Design is the intentional and thoughtful creation of solutions that address a specific problem or need, while also considering aesthetics, functionality, and user experience.
  
 design usually has a specific life cycle and some environment in its lifecycle.
  
 The first stage of the life cycle of design is Research. In this stage, the environment is classified as the following:
In the built environment of design research, designers gather information, analyze data, and explore potential solutions. 
In the human environment of design research, designers collaborate with stakeholders, gather feedback, and incorporate user perspectives. 
In the nature environment of design research, designers consider factors such as climate, geography, and natural resources when developing sustainable solutions.
  
 The second stage of the life cycle of design is Design. In this stage, the environment is classified as the following:
In the built environment, designers create sketc

**Third_stage: Identifying Interactions**

In [ ]:
def active_verb_extraction(text):

  #Extracting action verbs
    system_template1='''An action verb, also known as a dynamic verb, is a type of verb that describes an action or something that a person or thing does (e.g., “I  run”).
    Action verbs differ from stative verbs, which describe a state of being (e.g., “believe,” “want”, "is", "are", "am", "have", "has", "become", "feel").
    Note: Gerund (verb +ing) are not action verb. While run is an action verb, runing is not an action verb.
    You must return nothing if there is no action verb in the sentence.
    As an AI research assistant who can analyze sentences, your task is to identify action verbs in a sentence'''
    system_message_prompt1= SystemMessagePromptTemplate.from_template(system_template1)
    human_template1='Please return the action verbs in this text:\n {text}'
    human_message_prompt1=HumanMessagePromptTemplate.from_template(human_template1)
    prompt1=ChatPromptTemplate.from_messages([system_message_prompt1,human_message_prompt1])
    chain1=LLMChain(llm=llm, prompt=prompt1, output_key='action_verbs')

#Splitting text to the sentences, each sentence has one action verb
    system_template2= '''As an AI assistant,You would get some action verbs.
    Your task is to identify these action verbs in the text and split the text to independent sentences.
    Each sentence has only one active verb.
    You must return nothing if there is no action verb in the sentence.
    Your output is only a sentence without extra explaination.
    '''
    system_message_prompt2= SystemMessagePromptTemplate.from_template(system_template2)
    human_template2='''Regard these action verbs:\n {action_verbs}.\n
    In the following text, identify these action verbs and return a sentence with this format: subject + action verb + object.
    This is the text:\n {text}'''
    human_message_prompt2=HumanMessagePromptTemplate.from_template(human_template2)
    prompt2=ChatPromptTemplate.from_messages([system_message_prompt2,human_message_prompt2])
    chain2=LLMChain(llm=llm, prompt=prompt2, output_key='active_sentence')

#Creating the chain
    seq_chain=SequentialChain(chains=[chain1,chain2], input_variables=['text'],output_variables=['action_verbs','active_sentence'],verbose=False)

#Since we use the period sign to end sentences, we can split the text into sentences using this sign
    sentences = text.split(".")
    active_sentences_text=''

# Inputting each sentence to the chain created earlier
    for l in sentences:
       out_put=seq_chain(l)
       active_sentences_text=active_sentences_text + f"{out_put['active_sentence']}\n"

    print(f"the first stage is: \n {active_sentences_text} \n END \n")





In [ ]:
text='''
pool definition: A pool is a man-made body of water typically used for swimming, recreation, or water sports.

 pool usually has a specific life cycle and some environment in its lifecycle.

 The first stage of the life cycle of pool is Design and Planning. In this stage, the environment is classified as the following:
Built environment: The pool design includes the layout, materials, and features that will shape the overall look and functionality of the pool.
Human environment: Pool planners collaborate with architects, engineers, and clients to ensure the design meets all requirements and preferences.
Nature environment: The pool design takes into account factors such as sunlight exposure, wind patterns, and landscaping to create a harmonious connection with the natural surroundings.

 The second stage of the life cycle of pool is Construction. In this stage, the environment is classified as the following:
Built environment: Construction workers are busy excavating the ground, pouring concrete, and installing the pool's framework.
Human environment: Engineers, architects, and laborers are collaborating to ensure the pool is built according to design specifications.
Nature environment: The pool construction site is exposed to the elements, with sunlight and rain affecting the progress of the project.

 The third stage of the life cycle of pool is Maintenance. In this stage, the environment is classified as the following:
Built environment: The pool's tiles are being scrubbed clean, the filters are being replaced, and the water is being tested for pH levels.
Human environment: Pool technicians are working diligently to ensure the pool is safe and clean for swimmers.
Nature environment: The sun is shining down on the pool, helping to evaporate any excess water and keep the pool at a comfortable temperature.

 The fourth stage of the life cycle of pool is Renovation. In this stage, the environment is classified as the following:
Built environment: The pool is being drained, cleaned, and repaired by maintenance workers during the renovation process.
Human environment: Pool designers, contractors, and workers are collaborating to ensure the renovation project is completed efficiently and effectively.
Nature environment: The pool water is being treated with chemicals to maintain its quality and clarity during the renovation.

swimming definition: Swimming is the act of propelling oneself through water using the arms and legs in a coordinated motion.

 swimming usually has a specific life cycle and some environment in its lifecycle.

 The first stage of the life cycle of swimming is Learning to swim. In this stage, the environment is classified as the following:
Built environment: The swimming pool provides a safe and controlled space for beginners to practice their strokes and techniques.
Human environment: Instructors play a crucial role in guiding and teaching new swimmers, ensuring they develop confidence and skills in the water.
Nature environment: The temperature and cleanliness of the water impact the comfort and overall experience of those learning to swim.

 The second stage of the life cycle of swimming is Practicing swimming techniques. In this stage, the environment is classified as the following:
Built environment: Swimmers utilize lane ropes and starting blocks to enhance their technique during practice sessions.
Human environment: Coaches provide guidance and feedback to swimmers to help them improve their strokes and turns.
Nature environment: Swimmers feel the refreshing sensation of the water against their skin as they glide through the pool during practice.

 The third stage of the life cycle of swimming is Competing in swimming competitions. In this stage, the environment is classified as the following:
Built environment: Swimmers dive into the pool, push off the walls, and reach for the finish line in swimming competitions.
Human environment: Athletes, coaches, officials, and spectators come together to support and participate in swimming competitions.
Nature environment: Swimmers feel the cool water against their skin, hear the sound of splashing, and breathe in the fresh air while racing in swimming competitions.

 The fourth stage of the life cycle of swimming is Maintaining swimming skills. In this stage, the environment is classified as the following:
Built environment: Swimmers utilize swimming pools, lanes, starting blocks, and touchpads to practice and maintain their skills.
Human environment: Coaches, teammates, and spectators provide support, guidance, and motivation during swimming practice and competitions.
Nature environment: Swimmers are influenced by factors such as water temperature, weather conditions, and natural light when training in open water settings.

uninterrupted long distance swimming definition: Uninterrupted long distance swimming refers to the act of continuously swimming for an extended period of time without stopping, typically covering a significant distance in open water.

 uninterrupted long distance swimming usually has a specific life cycle and some environment in its lifecycle.

 The first stage of the life cycle of uninterrupted long distance swimming is Preparation. In this stage, the environment is classified as the following:
In the built environment, swimmers prepare their gear such as swimsuits, goggles, and swim caps.
In the human environment, coaches provide guidance and support to swimmers during their preparation for uninterrupted long distance swimming.
In the nature environment, swimmers assess the water conditions, temperature, and weather forecasts to plan their training sessions effectively.

 The second stage of the life cycle of uninterrupted long distance swimming is Training. In this stage, the environment is classified as the following:
Built environment: The training pool is equipped with lane lines and starting blocks to facilitate practice sessions for uninterrupted long distance swimming.

Human environment: Coaches provide guidance and support to swimmers during training sessions for uninterrupted long distance swimming.

Nature environment: Swimmers are exposed to varying water temperatures and weather conditions during their training for uninterrupted long distance swimming.

 The third stage of the life cycle of uninterrupted long distance swimming is Execution. In this stage, the environment is classified as the following:
Built environment: The swimmer's gear, such as swimsuit, goggles, and swim cap, is essential for the execution of uninterrupted long distance swimming.
Human environment: The presence of coaches, support crew, and fellow swimmers plays a crucial role in motivating and guiding the swimmer during the execution of uninterrupted long distance swimming.
Nature environment: The water temperature, currents, and weather conditions directly impact the swimmer's performance and experience during the execution of uninterrupted long distance swimming.

 The fourth stage of the life cycle of uninterrupted long distance swimming is Recovery. In this stage, the environment is classified as the following:
Built environment: The recovery stage of uninterrupted long distance swimming involves the use of recovery tools such as foam rollers and massage balls.
Human environment: During the recovery stage, swimmers may seek guidance from coaches or support from fellow swimmers to aid in their recovery process.
Nature environment: Swimmers may take advantage of natural elements such as cold water immersion or exposure to sunlight to enhance their recovery after long distance swimming.

designing definition: Designing is the process of creating solutions that address a specific problem or need, incorporating elements of creativity, functionality, and aesthetics.

 designing usually has a specific life cycle and some environment in its lifecycle.

 The first stage of the life cycle of designing is Research. In this stage, the environment is classified as the following:
In the built environment of designing, researchers gather data and analyze information to inform the design process.
In the human environment of designing, designers collaborate with stakeholders to understand their needs and preferences.
In the nature environment of designing, designers consider factors such as climate and natural resources when creating sustainable designs.

 The second stage of the life cycle of designing is Design. In this stage, the environment is classified as the following:
In the built environment of designing, architects create blueprints and engineers construct physical structures.
In the human environment of designing, designers collaborate with clients to understand their needs and preferences.
In the nature environment of designing, natural light and ventilation are considered to create sustainable and eco-friendly designs.

 The third stage of the life cycle of designing is Development. In this stage, the environment is classified as the following:
In the built environment of designing, architects create blueprints and engineers develop prototypes to bring the design concept to life.
In the human environment of designing, designers collaborate with clients, stakeholders, and end-users to ensure the design meets their needs and expectations.
In the nature environment of designing, designers consider factors such as natural light, ventilation, and landscaping to create sustainable and harmonious designs.

 The fourth stage of the life cycle of designing is Implementation. In this stage, the environment is classified as the following:
In the built environment of designing, materials are sourced and assembled to bring the design to life.
In the human environment of designing, collaboration and communication among team members play a crucial role in implementing the design effectively.
In the nature environment of designing, factors such as natural light, ventilation, and climate influence the design implementation process.

Swimmers cannot always use oceans and lakes to swim.
In bad weather, training can become impossible.
In a usual 50-meter pool the swimmer inevitably reaches the edge and has to turn around and push off to continue swimming, which impairs his technique and disrupts his rhythm.


'''

In [ ]:
active_verb_extraction(text)

the first stage is: 
 A pool is a man-made body of water typically used for swimming.
pool usually has a specific life cycle
The first stage of the life cycle of pool is Design.  
The first stage of the life cycle of pool is Planning.
The pool design includes the layout.  
The pool design will shape the overall look and functionality of the pool.
Pool planners collaborate with architects, engineers, and clients.  
Pool planners ensure the design meets all requirements and preferences.
The pool design takes into account factors such as sunlight exposure.  
The pool design creates a harmonious connection with the natural surroundings.
Construction.
Construction workers are excavating the ground.  
Construction workers are pouring concrete.  
Construction workers are installing the pool's framework.
Engineers, architects, and laborers are collaborating. 
Engineers, architects, and laborers are ensuring the pool is built.
The pool construction site is exposed to the elements.  
Sunlight an

**Fourth Stage: Establishing Functional analysis**

In [ ]:
# To create a functional analysis, we would utilize JSON to generate structured output.
json_schema2={'title':'Functional analysis',
             'description':'Identfying tools, objcets and their interactions',
             'type':'object',
             'properties':{
                 'Tool': {'title':'tool','description':'The objcet that change another object or protect another object to be changed','data_type':'string'},
                 'Object': {'title':'Object','description':'The Object which is changed or being protected form a change','data_type':'string'},
                 'How tool affect the object': {'title':'How tool affect on the object','description':'How tool affect on the object','data_type':'string'},
                 'How object affect the tool': {'title':'How object affect the tool','description':'How object affect the tool','data_type':'string'},
             },

             'required':['Tool','Object','How tool affect the object','How object affect the tool']
             }

In [ ]:
def functional_analysis(text):
  #Defining the tools and objects for the LLM, and extracting the relationship among them from each interaction.
  system_template='''you are an AI assistant that could analyze a sentence and specify tool, object and the impact of tools on object and the impact of object on tool'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)

  text1= 'Alex write a sentence'
  example_one= HumanMessagePromptTemplate.from_template(text1)
  text1_out_put= 'Tool: Alex, Object: Sentence, How tool affect the object: creating the sentence, How object affect the tool:None'
  example_one_output= AIMessagePromptTemplate.from_template(text1_out_put)

  text2='Ali eats a sandwich'
  example_two= HumanMessagePromptTemplate.from_template(text2)
  text2_out_put= 'Tool: Ali, Object: Sandwich,  How tool affect the object: consuming the sandwich, How object affect the tool:None'
  example_two_output= AIMessagePromptTemplate.from_template(text2_out_put)

  text3='Dog walks near the school'
  example_three= HumanMessagePromptTemplate.from_template(text3)
  text3_out_put= 'specified action means tool does something that impact the object. There is imapct on dog or school. Thus there is no tool, and object'
  example_three_output= AIMessagePromptTemplate.from_template(text3_out_put)

  text4='The manager interview a job applicant'
  example_four= HumanMessagePromptTemplate.from_template(text4)
  text4_out_put= 'Tool: Job applicant, Object: Manger,  How tool affect the object: increasing the information of manager, How object affect the tool:None '
  example_four_output= AIMessagePromptTemplate.from_template(text4_out_put)

  Human_prompt= '''In every action there is a tool and object. Action means tool does something that causes object to change or to be maintained from the change.
  Thus in every action there are two material object (one is tool and the other is object) that are interacting with each other.
  In This interaction the tool must impact the object. If there is no impact on the object, we can conclude that there is no action.
  The object that impact on the other one is called as a tool and the one that is got affected is the bject.
  Please consider these sentences:\n {text} \n please clarify waht are tools and objects in every sentence.
  Please also clarify how the tool affects the object and how the object affect the tool'''
  prompt= HumanMessagePromptTemplate.from_template(Human_prompt)
  chat_prompt=ChatPromptTemplate.from_messages([prompt,system_message_prompt])

  # Creating the structured out put with the json defined earlier
  chain=create_structured_output_chain(json_schema2,llm, chat_prompt,verbose=False)

  #To make it easier to use, we will input all the interactions into the function. Using the split() method, we will separate each sentence (interaction) and use LLM to consider each interaction separately.
  sentences = text.split(".")
  result=[]

#Executing the chain
  for l in sentences:
   out_put=chain.run(text=l)
   print(f"\n the sentence is: \n {l}")
   print(f" the functional analysis is: \n {out_put}")
   result.append(out_put)

In [ ]:
text='''
Active sentences:
I finished high school.
I entered the university there.
I went to military service.
I worked for a year.
I accepted as a student in Canada.
I came to Concordia university.
I do a PhD program.
I am working for a professor at Concordia University.
I go to some cultural centers.
I voluntarily work with them.
I am using social applications.
I got injured on my back.
I must exercise to relieve my backache.
I would work with Python.
I have a part-time job.
I meet a few people.
I have to finish my degree.
I find a job.
You have to understand it.
You have to talk to her.
You can use romantic restaurants.
She can spend quality time with her partner in beautiful parks.
You have to use compliments.
The girl is influenced by her friends and family members.
You need to buy some gifts.
You have to date with her more and more.
She gets more committed to you.
To find new places, ideas, or possibilities through curiosity and adventurousness.
We need some friends.
We ask humans.
We can also ask other humans in the specific environment about a subject.
After discovering, we have to evaluate the thing.
We can use others' experience to evaluate the founded object.
You can use search engines.
You can use maps.
You can investigate something.
I can find a girlfriend.
The girls go.
Girls go to university.
Girls go to parties.
Girls go to the park.
Girls go to the museum.
Girls go to offices.
Girls work.
They would go to classes.
They would attend workshops.
They would go to Gyms.
They would go to cafes.
They would go to shopping centers.
They use online platforms.
They would go to travel.
They would go to libraries.
I go to where the girls are.

'''

In [ ]:
functional_analysis(text)


 the sentence is: 
 
Active sentences:
I finished high school
 the functional analysis is: 
 {'Tool': 'I', 'Object': 'high school', 'How tool affect the object': 'finishing', 'How object affect the tool': 'achievement'}

 the sentence is: 
 
I entered the university there
 the functional analysis is: 
 {'Tool': 'I', 'Object': 'university', 'How tool affect the object': 'Entering', 'How object affect the tool': 'Providing education'}

 the sentence is: 
 
I went to military service
 the functional analysis is: 
 {'Tool': 'I', 'Object': 'military service', 'How tool affect the object': 'I went to military service', 'How object affect the tool': 'Military service requires the presence and participation of individuals like me'}

 the sentence is: 
 
I worked for a year
 the functional analysis is: 
 {'Tool': 'I', 'Object': 'work', 'How tool affect the object': 'Performed the action of working', 'How object affect the tool': 'Provided the opportunity to work'}

 the sentence is: 
 
I accep

**Fifth Stage: Idea Generation with TRIZ tools**

In [ ]:
#So far, all the temperature settings for each section and function have been at 0 to produce consistent and predictable output. However, we have now increased the temperature setting to 1 to enhance the creativity of GPT and generate better ideas in terms of both quality and quantity.
llm_h= ChatOpenAI(openai_api_key=openAI_API_Key,temperature=1)

In [ ]:
def TRIZ_principles(problem, contradiction, principles):

  #Asking LLm to generate ideas for each TRIZ principle
  system_template='''you are an AI assistant that could get a design problem and its contradiction and generate ideas regarding TRIZ principles.
  The out-put text should clearly state generated ideas and provide an example for each idea to better explain them.
  Your out-put is only the ideas and examples witout extra explaination.'''
  system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)

  Human_prompt= '''TRIZ is an acronym for "Theory of Inventive Problem Solving."
   It is a problem-solving methodology that originated in Russia and was developed by Genrich Altshuller and his colleagues in the mid-20th century.
   The 40 Principles of TRIZ are a set of guidelines formulated by Genrich Altshuller, the founder of TRIZ.
   These principles are based on his analysis of thousands of patents across various industries.
   They serve as strategies for overcoming contradictions and generating inventive solutions for problem-solving and innovation.
   According to TRIZ in every design problem there is a contradiction that would make this design problme.
   Now, I would like to give you a design problem with its contradiction. Please resolve it using Principle {number} of TRIZ.
   \n Here's the problem:\n{problem}
   \n This is the contradiction recognized in the problem:\n {contradiction}.
   '''
  prompt= HumanMessagePromptTemplate.from_template(Human_prompt)
  chat_prompt=ChatPromptTemplate.from_messages([prompt,system_message_prompt])
  ideas=''
  Principle_number=principles
  for x in range(Principle_number):
    model_request= chat_prompt.format_prompt(number= str(x+1),problem=problem, contradiction=contradiction).to_messages()
    result=llm_h(model_request)
    print(f"\n{result.content}")

    #saving all the generated ideas in a list. This list usually includes more than 100 ideas.
    ideas= ideas + result.content



    # Splitting the list of ideas to shorter chunck and removing duplicated ideas in each chunk
    text_splitter=CharacterTextSplitter.from_tiktoken_encoder(chunk_size=600)
    splitted_ideas=text_splitter.split_text(ideas)



    system_template='''
   You are an AI assistant who is an expert in removing repetitive ideas.
   I will provide you with a list of ideas and their examples.
   Some of these ideas may be repetitive, so please remove any duplicates and return a list of unique ideas with their examples.
  '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= 'Please consider the list of ideas and only return a list of unique ideas. \n list of ideas: \n {ideas} '
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    summerized_ideas= ''

    for k in splitted_ideas:
     request=prompt0.format_prompt(ideas=k).to_messages()
     summerized_ideas = summerized_ideas + llm(request).content




  #Removing duplicated ideas for the second time
    system_template='''
   You are an AI assistant who is an expert in removing repetitive ideas.
   I will provide you with a list of ideas and their examples.
   Some of these ideas may have repetitive concept, so please remove ideas with the same concepts and return a list of unique ideas with their examples.
   '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= 'Please consider the list of ideas and only return a list of unique ideas. \n list of ideas: \n {summerized_ideas} '
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    request2=prompt0.format_prompt(summerized_ideas=summerized_ideas).to_messages()
    second_summerized_ideas=llm(request2).content



    # LLM is requested to consider the design problem, review the generated ideas, and suggest team of experts for more consultation.
    system_template='''
   You are an AI assistant who is an expert to detremine which experts are required in a design team for resolving a design problem.
   you get the design problem, contradiction in the design problem and generated ideas.
   You would return the experts needed to validate and implement the ideas to resolve the design problem and create a novel product.
   '''
    system_message_prompt= SystemMessagePromptTemplate.from_template(system_template)
    human_template= '''this is the design problem: \n {problem}
    \n\n this is the contradiction inside the design problem: {contradiction}
    \n\n this is the generated ideas:{final_ideas}
    '''
    human_message_prompt=HumanMessagePromptTemplate.from_template(human_template)
    prompt0=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])

    request3=prompt0.format_prompt(problem=problem, contradiction=contradiction, final_ideas=second_summerized_ideas).to_messages()
    design_team=llm(request3).content





#printing the results
  return print(f" \n \n This is the contradiction that we face:\n{contradiction}.\n \n \n Regarding TRIZ principles from number 1 to number {Principle_number}, we would have these ideas:\n\n{summerized_ideas} \n\n\n--------------\n\n\n second_summerized ideas:\n{second_summerized_ideas} \n\n\n These experts can help you: \n{design_team}")







In [ ]:
problem='''Design a pool suitable for uninterrupted long distance swimming'''
contradiction='''A swimming line has an end, But We must not have an end because it would destroy the rhythm of the swimmer for uninterrupted long-distance swimming'''

In [ ]:
TRIZ_principles(problem=problem, contradiction=contradiction, principles=15)


Principle 1: Segmentation - Divide an object into independent parts
   Example: Create a pool with separate lanes that loop around, allowing swimmers to continue swimming without reaching an end.

Principle 1: Segmentation - Introduce a temporary solution or intermediate state
   Example: Install underwater mirrors at the end of the pool to give the illusion of an endless swim, helping swimmers maintain their rhythm.

Principle 1: Segmentation - Use multiple smaller pools connected in a loop formation
   Example: Design a series of smaller pools linked together to form a continuous circuit for long-distance swimming without a definitive end.

Principle 2: Taking out provides: 
-Design a pool with an endless swimming lane looped back on itself, allowing swimmers to continuously swim without reaching an end.

Principle 2: Segmentation:
-Design a pool with multiple connected lanes that can be segmented n such a way that swimmers can move from one lane to another seamlessly without interr